In [19]:
import faiss
import torch
print(torch.cuda.is_available())

from uuid import uuid4
from pymongo import MongoClient
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

True


In [20]:
import sys
sys.path.append("/culture_backoffice_data/app")

from app.src.v1.db.mongo import connect_to_mongo, convert_objectid_to_str
from app.src.v1.rag.retriever.vector_retriever import VectorRetriever
from app.src.v1.rag.retriever.hybrid_retriever import HybridRetriever

from notebooks.eda import analyze_document_lengths, analyze_token_lengths

In [21]:
embed_model_name = "intfloat/multilingual-e5-large-instruct"

## Data

### DB에서 데이터 꺼내기

In [22]:
company_collection = connect_to_mongo("culture_db", "company")
hompage_collection = connect_to_mongo("culture_db", "company_homepage")

# 기업 데이터 가져오기
print("기업 데이터 불러오는 중...")
companies = company_collection.find()
total_data = []
for company in companies:
    id = company["_id"]
    name = company["company_name"]
    biz_no = company["biz_no"]
    total_data.append((id, name, biz_no))

print(f"총 {len(total_data)}개 기업 데이터를 불러왔습니다.")

기업 데이터 불러오는 중...
총 60개 기업 데이터를 불러왔습니다.


### 길이 분석

In [23]:
doc_results = analyze_document_lengths(total_data, hompage_collection)

기업별 문서 길이 분석 중...


100%|██████████| 60/60 [00:00<00:00, 569.18it/s]

In [24]:
token_results = analyze_token_lengths(total_data, hompage_collection, embed_model_name)

임베딩 모델 'Alibaba-NLP/gte-multilingual-base'의 토크나이저로 토큰화 분석 중...


100%|██████████| 60/60 [00:14<00:00,  4.16it/s]
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'defaul

In [25]:
print("\n분석이 완료되었습니다.")
print(f"분석 결과는 'analysis_results' 디렉토리에 저장되었습니다.")

# 요약 정보 출력
print("\n=== 문서 길이 분석 요약 ===")
print(f"총 문서 수: {doc_results['total_stats']['total_documents']}")
print(f"평균 문서 길이: {doc_results['total_stats']['avg_length']:.1f} 글자")
print(f"중앙값 문서 길이: {doc_results['total_stats']['median_length']:.1f} 글자")
print(f"최소 문서 길이: {doc_results['total_stats']['min_length']} 글자")
print(f"최대 문서 길이: {doc_results['total_stats']['max_length']} 글자")

print("\n=== 토큰 길이 분석 요약 ===")
print(f"모델: {embed_model_name}")
print(f"평균 토큰 길이: {token_results['total_token_stats']['avg_token_length']:.1f} 토큰")
print(f"중앙값 토큰 길이: {token_results['total_token_stats']['median_token_length']:.1f} 토큰")
print(f"최소 토큰 길이: {token_results['total_token_stats']['min_token_length']} 토큰")
print(f"최대 토큰 길이: {token_results['total_token_stats']['max_token_length']} 토큰")
print(f"평균 토큰/문자 비율: {token_results['correlation_data']['token_to_char_ratio'].mean():.4f}")


분석이 완료되었습니다.
분석 결과는 'analysis_results' 디렉토리에 저장되었습니다.

=== 문서 길이 분석 요약 ===
총 문서 수: 7528
평균 문서 길이: 2357.2 글자
중앙값 문서 길이: 1467.0 글자
최소 문서 길이: 11 글자
최대 문서 길이: 116366 글자

=== 토큰 길이 분석 요약 ===
모델: Alibaba-NLP/gte-multilingual-base
평균 토큰 길이: 905.0 토큰
중앙값 토큰 길이: 592.5 토큰
최소 토큰 길이: 6 토큰
최대 토큰 길이: 42343 토큰
평균 토큰/문자 비율: 0.4460


In [26]:
hompage_collection = connect_to_mongo("culture_db", "company_homepage")

total_pages = []
for id, name, biz_no in total_data:
    id = convert_objectid_to_str(id)
    print(f"{id}, {name} {biz_no}")
    homepage = hompage_collection.find_one({"company_id": id})

    if homepage is None:
        print(f"Warning: No homepage data found for {name}")
        continue

    pages = homepage["pages"]
    print(f"{name} : {len(pages)}")
    for page in homepage["pages"]:
        total_pages.append((name, page))

print(len(total_pages))

67d14af523c5f11bad8c2f62, 엘지씨엔에스 116-81-19477
엘지씨엔에스 : 116
67d14af523c5f11bad8c2f64, 지에스피앤엘 466-87-02854
지에스피앤엘 : 24
67d14af523c5f11bad8c2f65, 엠앤씨솔루션 263-81-01759
엠앤씨솔루션 : 171
67d14af523c5f11bad8c2f66, 더본코리아 211-86-00870
더본코리아 : 271
67d14af523c5f11bad8c2f67, 한화비전 477-88-01018
한화비전 : 500
67d14af523c5f11bad8c2f68, 에이치에스효성 774-86-03325
에이치에스효성 : 30
67d14af523c5f11bad8c2f69, 에이치디현대마린솔루션 582-81-00535
67d14af523c5f11bad8c2f6a, 포스코디엑스 219-81-00428
포스코디엑스 : 69
67d14af523c5f11bad8c2f6b, 두산로보틱스 257-88-00128
67d14af523c5f11bad8c2f6c, 에스케이오션플랜트 615-81-06111
에스케이오션플랜트 : 6
67d14af523c5f11bad8c2f6d, 쏘카 616-81-90529
쏘카 : 220
67d14af523c5f11bad8c2f6e, 카카오페이 527-88-00686
카카오페이 : 61
67d14af523c5f11bad8c2f6f, 에이치디현대중공업 252-87-01412
에이치디현대중공업 : 501
67d14af523c5f11bad8c2f70, 크래프톤 220-87-45316
67d14af523c5f11bad8c2f71, 에스케이바이오사이언스 818-86-00991
에스케이바이오사이언스 : 55
67d14af523c5f11bad8c2f73, 케이씨씨글라스 401-87-01412
케이씨씨글라스 : 127
67d14af523c5f11bad8c2f74, 한화시스템 513-81-17175
한화시스템 : 232
67d14af523c5f11bad8c2f75, 현대오토에버

In [27]:
total_docs = []
for company_name, page in total_pages:
    doc_id = str(uuid4())
    total_docs.append(
        Document(
            page_content=page["text"], 
            metadata={
                "company_name": company_name, 
                "url": page["url"],
                "original_doc_id": doc_id
            }, 
        id=doc_id)
    )

print(len(total_docs))

7528


### 문자열 청킹

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],
    chunk_size=2000,
    chunk_overlap=250,
    length_function=len
)

chunks = text_splitter.split_documents(total_docs)
print(len(chunks))

15411


In [29]:
import random

sample_idx = random.randint(0, len(total_docs))
print(sample_idx)

2371


In [30]:
original_docs_by_id = {}
for doc in total_docs:
    original_docs_by_id[doc.metadata['original_doc_id']] = doc

# 청크에서 원본 문서 찾기 함수
def find_original_document(chunk):
    """청크에서 원본 문서 찾기"""
    if 'original_doc_id' in chunk.metadata:
        original_id = chunk.metadata['original_doc_id']
        return original_docs_by_id.get(original_id)
    return None

In [31]:
sample_chunk = chunks[sample_idx]
print(sample_chunk.metadata['original_doc_id'])
print(sample_chunk.metadata['company_name'], sample_chunk.metadata['url'])
print("=" * 100)
print(len(sample_chunk.page_content))
print(sample_chunk.page_content)

print("\n\n")

original_doc = find_original_document(sample_chunk)
print(original_doc.metadata['company_name'], original_doc.metadata['url'])
print("=" * 100)
print(len(original_doc.page_content))
print(original_doc.page_content)

735b1602-71d5-4916-93fa-3335d2833e90
카카오페이 https://kakaopay.com/terms/service
1974
제 22 조의 3 (카드모집서비스 관련 자료 열람권)
① 회원은 『금융소비자 보호에 관한 법률』에 따른 분쟁조정 또는 소송의 수행 등 권리구제를 위한 목적으로 회사 또는 제휴카드사에게 이들이 같은 법 제 28 조에 따라 보관하는 자료의 열람(사본의 제공 또는 청취를 포함)을 요구할 수 있습니다. 다만, 신용카드 이용계약의 체결, 이행, 청약철회, 해지 등 회원과 제휴카드사 사이의 계약에 관한 자료는 제휴카드사에 요구하여야 합니다.
② 회사는 회원이 제 1 항에 따른 자료의 열람을 요구한 날부터 8 일 이내에 해당 자료를 열람할 수 있도록 해당 회원에게 제공합니다. 다만, 회사는 해당 기간 내에 자료를 제공할 수 없는 정당한 사유가 있을 때에는 회원에게 그 사유를 알리고 열람을 연기할 수 있습니다.
③ 회사는 회원의 제 1 항에 따른 자료의 열람 요구에 대하여, 다음 각 호의 어느 하나에 해당하는 경우에는 회원에게 그 사유를 알리고 열람을 제한하거나 거절할 수 있습니다.
가. 관련 법령에 따라 열람을 제한하거나 거절할 수 있는 경우
나. 다른 사람의 생명∙신체를 해칠 우려가 있거나 다른 사람의 재산과 그 밖의 이익을 부당하게 침해할 우려가 있는 경우
다. 회사 또는 제휴카드사의 영업비밀을 침해할 우려가 있는 경우
라. 개인정보의 공개로 인해 사생활의 비밀 또는 자유를 부당하게 침해할 우려가 있는 경우
마. 열람하려는 자료가 열람목적과 관련이 없다는 사실이 명백한 경우
④ 본 조의 자료 열람권은 『금융소비자 보호에 관한 법률』의 적용을 받는 카드모집서비스의 신용카드상품에 한하여 적용됩니다.
제 22 조의 4 (카드모집서비스 관련 분쟁처리 절차)
① 회원은 카드모집서비스 이용과 관련하여 회사에 이의가 있는 경우 관련 법령에 따라 서면(전자문서 포함) 또는 인터넷 등을 통하여 회사에 이의를 제기할 수 있습니다. 회사의 분

### 토큰 청킹

In [32]:
# from transformers import AutoTokenizer

# def hf_tokenizer_len(text):
#     # 원하는 모델에 맞는 토크나이저 선택 (예: BERT, RoBERTa 등)
#     tokenizer = AutoTokenizer.from_pretrained(embed_model_name)
#     tokens = tokenizer.encode(text)
#     return len(tokens)

# text_splitter = RecursiveCharacterTextSplitter(
#     separators=["\n\n", "\n", ". ", " ", ""],
#     chunk_size=900,
#     chunk_overlap=150,
#     length_function=hf_tokenizer_len  # 토큰 길이 계산 함수
# )

In [33]:
# token_chunks = text_splitter.split_documents(total_docs)
# print(len(token_chunks))

In [34]:
# from langchain_experimental.text_splitter import SemanticChunker
# from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings(
#     model_name=embed_model_name,
#     model_kwargs=cfg["model_kwargs"],
#     encode_kwargs=cfg["encode_kwargs"]
# )

# semantic_splitter = SemanticChunker(
#     embeddings=embeddings,
#     breakpoint_threshold_type="percentile",
#     breakpoint_threshold_amount=70,
# )

# # 시맨틱 청킹 적용
# semantic_chunks = semantic_splitter.split_documents(total_docs)
# print(f"시맨틱 청킹 후 청크 개수: {len(semantic_chunks)}")

# # 샘플 시맨틱 청크 확인
# sample_semantic_idx = random.randint(0, len(semantic_chunks) - 1)
# sample_semantic_chunk = semantic_chunks[sample_semantic_idx]
# print(f"샘플 시맨틱 청크 {sample_semantic_idx}:")
# print("=" * 100)
# print(f"회사명: {sample_semantic_chunk.metadata['company_name']}")
# print(f"URL: {sample_semantic_chunk.metadata['url']}")
# print(f"길이: {len(sample_semantic_chunk.page_content)} 글자")
# print("=" * 100)
# print(sample_semantic_chunk.page_content)

## Vector Retriever

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# embedding_model = UpstageEmbeddings(model="upstage-e5-large")

In [41]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {
    "device": "cuda:1",
    "trust_remote_code": True
}
encode_kwargs = {
    "normalize_embeddings": True,
    "batch_size": 128,
    "multi_process": True,
    "show_progress": True
}

embedding_model = HuggingFaceEmbeddings(
    model_name=embed_model_name, 
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [42]:
from langchain_core.vectorstores import InMemoryVectorStore

"""
벡터 저장소에 만들어질 index 타입을 설정.
 - IP : 내적을 기반으로 코사인 유사도 계산으로 쿼리 벡터와 벡터 저장소 내 모든 벡터를 비교하여 가장 유사한 벡터를 찾는다.
 - L2 : 유클리드 거리를 기반으로 유사도 계산으로 쿼리 벡터와 벡터 저장소 내 모든 벡터를 비교하여 가장 유사한 벡터를 찾는다.
 - HNSW : 비정형 데이터에 적합한 효율적인 검색을 위한 인덱스 타입으로 모든 벡터를 비교하지 않고 적절한 후보 벡터를 찾아 비교하여 가장 유사한 벡터를 찾는다.
"""
index = faiss.IndexFlatIP(len(embedding_model.embed_query(sample_chunk.page_content)))
## index = faiss.IndexFlatL2(len(embedding_model.embed_query(sample_text)))
## index = faiss.IndexHNSWFlat(len(embedding_model.embed_query(sample_text)), 32)

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(chunks)

['5e302056-ba7b-4bcf-acfc-3437be74b297',
 '0b59b85f-6f90-47be-aa9d-2e3e930752ab',
 'ad25624d-1c0e-447f-8b46-d698974aee0d',
 'd317fbba-0b3e-440e-8ffc-613138a9cb4d',
 '634b872d-bfda-4f91-9f42-0d8d7625de0a',
 '031a8d98-3091-48d5-98bc-f76d61ee8746',
 'ac7398de-7313-46fe-a08b-046c9c5feb95',
 '7d7182db-934d-40c8-be3a-fe8380411360',
 'b24b25ac-1048-4e74-bce0-4bb5ee06241a',
 'beb6870c-c6ba-4b5c-9fdc-5dd11abf46f0',
 '85996371-5336-4865-9dad-5a215fa047d0',
 '588680ac-070a-471d-913b-03064aef3177',
 'a9df2d14-941b-4e62-93f5-1a8343fb047a',
 '8b960855-642d-455e-8d4e-2d7102e205ed',
 'e86559ae-abae-471b-81f2-bc9cce7486b7',
 '4c186b18-c176-49ba-b0e9-5b55e3453886',
 '12742c90-471e-44a9-899e-7b1538d958d4',
 '622a6131-627e-4aec-8014-3104ad39bcee',
 'eccacf15-87f2-4753-8c4c-bcb7bf71734b',
 'aa0bc2d6-481f-4563-aed4-3f25733a7a3f',
 '8a1de3be-f793-42ef-ba9b-bd3ac1fdbccd',
 '43311b18-ed21-4342-be43-df85494a0917',
 'dc1222f6-233c-4a67-b834-935adc926b75',
 '2014b6c2-9d0b-4251-9d02-e679b3f1a4eb',
 '3ec0086b-34bb-

In [45]:
import os
from datetime import datetime

now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
save_path = f"./indexes/{now}"
os.makedirs(save_path, exist_ok=True)

vector_store.save_local(save_path)

In [43]:
ret_results = vector_store.similarity_search_with_score("코리아교육그룹의 비전, 목표") ## 쿼리에 대한 검색 수행

for document, score in ret_results:
    print(document.metadata['company_name'], document.metadata['url'], score) ## 회사명, 홈페이지 주소, 유사도 점수
    print("=" * 100)
    print(document.page_content) ## 본문 내용
    print("\n\n")

코리아교육그룹 http://www.koreaedugroup.com/2018/about/ceo.asp 0.9102703
EDUCATION
COMPANY
CAREERS
CEO MESSAGE
HISTORY
PERFORMANCE
EDUCATION SHARING
GROUP ACTIVITIES
오늘보다 더 나은 내일
A bright future for Korean education
FOR BETTER FUTURE
미래를 만드는 꿈의 공방
코리아교육그룹은 ‘미래를 만드는 꿈의 공방’을 모토로 교육이 곧 미래의 비전이라는 설립취지에 따라 교육서비스를 핵심사업으로 성장해 가고 있는 전문 기업입니다. 저희 코리아교육그룹은 10여 년 전 컴퓨터그래픽과 디자인 교육을 시작으로 온라인 원격교육과 뷰티관련 교육, 항공승무원 및 서비스인력 양성교육 등 다양한 교육서비스의 확장을 통해 전문 인력을 양성해 왔습니다.

코리아교육그룹 전 임직원들은 개인의 미래를 개척하고 더 나아가 국가의 미래를 이끌어 갈 인재를 양성한다는 사명감과 자부심으로 항상 새로운 이상과 목표를 향해 나아가고 있습니다. 무한 경쟁의 시대, 코리아교육그룹은 남과의 경쟁이 아닌 지금껏 걸어온 자신과의 경쟁이 우선임을 자각하고 높은 이상의 실현을 위해 일신우일신(日新又日新)하는 교육기업이 되겠습니다.

교육서비스의 더 나은 미래를 위해 전 임직원이 함께 움직이는 코리아교육그룹의 힘찬 발걸음을 성원하고 지켜봐 주시기 바랍니다. 감사합니다.
코리아교육그룹 대표이사
김영우
그룹소개
찾아오시는길
인재채용
서울 강남구 강남대로 286 부영빌딩 3, 4층 (우)06258 대표이사 김영우
© KOREA EDUCATION GROUP. All Rights Reserved.



코리아교육그룹 http://www.koreaedugroup.com/2018/about/share.asp 0.90940934
EDUCATION
COMPANY
CAREERS
CEO MESSAGE
HISTORY
PERFORMANCE
EDUCATION SHARING
GRO

In [44]:
ret_results = vector_store.similarity_search_with_relevance_scores("코리아교육그룹의 비전, 목표") ## 쿼리에 대한 검색 수행

for document, score in ret_results:
    print(document.metadata['company_name'], document.metadata['url'], score) ## 회사명, 홈페이지 주소, 유사도 점수
    print("=" * 100)
    print(document.page_content) ## 본문 내용
    print("\n\n")

코리아교육그룹 http://www.koreaedugroup.com/2018/about/ceo.asp 0.35634171676448
EDUCATION
COMPANY
CAREERS
CEO MESSAGE
HISTORY
PERFORMANCE
EDUCATION SHARING
GROUP ACTIVITIES
오늘보다 더 나은 내일
A bright future for Korean education
FOR BETTER FUTURE
미래를 만드는 꿈의 공방
코리아교육그룹은 ‘미래를 만드는 꿈의 공방’을 모토로 교육이 곧 미래의 비전이라는 설립취지에 따라 교육서비스를 핵심사업으로 성장해 가고 있는 전문 기업입니다. 저희 코리아교육그룹은 10여 년 전 컴퓨터그래픽과 디자인 교육을 시작으로 온라인 원격교육과 뷰티관련 교육, 항공승무원 및 서비스인력 양성교육 등 다양한 교육서비스의 확장을 통해 전문 인력을 양성해 왔습니다.

코리아교육그룹 전 임직원들은 개인의 미래를 개척하고 더 나아가 국가의 미래를 이끌어 갈 인재를 양성한다는 사명감과 자부심으로 항상 새로운 이상과 목표를 향해 나아가고 있습니다. 무한 경쟁의 시대, 코리아교육그룹은 남과의 경쟁이 아닌 지금껏 걸어온 자신과의 경쟁이 우선임을 자각하고 높은 이상의 실현을 위해 일신우일신(日新又日新)하는 교육기업이 되겠습니다.

교육서비스의 더 나은 미래를 위해 전 임직원이 함께 움직이는 코리아교육그룹의 힘찬 발걸음을 성원하고 지켜봐 주시기 바랍니다. 감사합니다.
코리아교육그룹 대표이사
김영우
그룹소개
찾아오시는길
인재채용
서울 강남구 강남대로 286 부영빌딩 3, 4층 (우)06258 대표이사 김영우
© KOREA EDUCATION GROUP. All Rights Reserved.



코리아교육그룹 http://www.koreaedugroup.com/2018/about/share.asp 0.3569504858443714
EDUCATION
COMPANY
CAREERS
CEO MESSAGE
HISTORY
PERFORMANCE
EDUCAT